## Get OpenSky Data

In [1]:
from dotenv import load_dotenv

In [2]:
if not load_dotenv():
    print("Failed to load .env file")

In [3]:
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime
import os

keys = [
    "icao24", "callsign", "origin_country", "time_position", "last_contact", "longitude", 
    "latitude", "geo_altitude", "on_ground", "velocity", "true_track", "vertical_rate",
    "sensors","baro_altitude", "squawk", "spi", "position_source", "category"
]

open_sky_user = os.getenv('OPENSKY_USER') or ''
open_sky_password = os.getenv('OPENSKY_PASS') or ''

auth = HTTPBasicAuth(open_sky_user,open_sky_password)

def retrieve_data() :
    url = "https://opensky-network.org/api/states/all"
    optional_params = {
        # 'lamin':48.3,
        # 'lomin':1.5,
        # 'lamax':49.2,
        # 'lomax':3.5
    }
    try:
        response = requests.get(url, auth = auth, params=optional_params)
        response.raise_for_status()
        flights = response.json()
        time = flights["time"]
        values = flights["states"]
        # Get current UTC date and time formated as string
        flight_time = datetime.fromtimestamp(time).isoformat()
        print(flight_time)

        # Recreate dictionary
        info_flights = [{keys[i]: values[j][i] for i in range(len(values[j]))} for j in range(len(values))]
        print("Data collected successfully!\n Number of flights: ", len(info_flights))
        return flight_time, info_flights

    except Exception as e:
        print(e, response.text)


In [4]:
retrieve_data()

2024-12-19T15:09:24
Data collected successfully!
 Number of flights:  8748


('2024-12-19T15:09:24',
 [{'icao24': 'a89ea5',
   'callsign': 'N6545H  ',
   'origin_country': 'United States',
   'time_position': 1734617363,
   'last_contact': 1734617363,
   'longitude': -97.8074,
   'latitude': 35.6626,
   'geo_altitude': 731.52,
   'on_ground': False,
   'velocity': 39.8,
   'true_track': 243.1,
   'vertical_rate': -0.33,
   'sensors': None,
   'baro_altitude': 746.76,
   'squawk': '7314',
   'spi': False,
   'position_source': 0},
  {'icao24': '4b1816',
   'callsign': 'SWR935L ',
   'origin_country': 'Switzerland',
   'time_position': 1734617363,
   'last_contact': 1734617363,
   'longitude': 13.6645,
   'latitude': 45.306,
   'geo_altitude': 10988.04,
   'on_ground': False,
   'velocity': 218.26,
   'true_track': 314.62,
   'vertical_rate': 0,
   'sensors': None,
   'baro_altitude': 11087.1,
   'squawk': '2055',
   'spi': False,
   'position_source': 0},
  {'icao24': '4b1819',
   'callsign': 'SWR2VK  ',
   'origin_country': 'Switzerland',
   'time_position': 17

## Send to Event Hub

In [5]:
%pip install azure-eventhub

Note: you may need to restart the kernel to use updated packages.


In [7]:
from azure.eventhub import EventData
from azure.eventhub.aio import EventHubProducerClient

import time
import asyncio
import json
import os

In [6]:
EVENT_HUB_CONNECTION_STR = os.getenv('EVENT_HUB_CONNECTION_STR') or ''
EVENT_HUB_NAME = os.getenv("EVENT_HUB_NAME") or '' #name of hub NOT THE NAMESPACE

In [11]:
async def send_data():
    # create a producer to send message to the event hub
    # specify a connection string to your event hubs namespace and the event hub name
    producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENT_HUB_CONNECTION_STR,
        eventhub_name=EVENT_HUB_NAME
    )
    
    async with producer:
        data = retrieve_data()
        if data is None:
            print("No data to send")
            return
    
        time_request = data[0]
        flights = data[1]
        if len(flights) <= 0:
            print("No flights data")
            return
        try:
            await send_batches(producer, time_request, flights)
        except:
            print("Error while sending data")

async def send_batches(producer:EventHubProducerClient, time_request:str, flights:list):
    #create a batch of events
    event_data_batch = await producer.create_batch()
    
    for element in flights:
        # Add events to the batch.
        element["time"] = time_request
        event_data = EventData(json.dumps(element).replace('None', ''))
        try:
            event_data_batch.add(event_data)
        except ValueError: # if the batch is full, send it, then create a new batch, and add the event to the new batch
            #send the batch of events to the event hub
            await producer.send_batch(event_data_batch)
            print(f"Batch sent: {time_request}, size: {len(event_data_batch)}")
            
            event_data_batch = await producer.create_batch()
            event_data_batch.add(event_data) # this event would be lost (not included in the previous batch) 
            
    #send the batch of events to the event hub
    await producer.send_batch(event_data_batch)
    print(f"Batch sent: {time_request}, size: {len(event_data_batch)}")

In [12]:
async def run_async(duration:float, frequency:float):
    end_time = time.time() + (60 * duration) # in seconds
    while time.time() < end_time:
        start_time = time.time()
        await send_data()
        waiting_time = 60/frequency
        processing_time = time.time() - start_time
        await asyncio.sleep(waiting_time - processing_time)

In [14]:
# import sys
# run_async(float(sys.argv[1]), float(sys.argv[2]))
await run_async(duration=2, frequency=1) # 2 minutes, 1 call per minute
# asyncio.run(run_async(duration=2, frequency=1))

2024-12-19T16:36:14
Data collected successfully!
 Number of flights:  10147
Batch sent: 2024-12-19T16:36:14, size: 613
Batch sent: 2024-12-19T16:36:14, size: 613
Batch sent: 2024-12-19T16:36:14, size: 611
Batch sent: 2024-12-19T16:36:14, size: 611
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 611
Batch sent: 2024-12-19T16:36:14, size: 613
Batch sent: 2024-12-19T16:36:14, size: 611
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 612
Batch sent: 2024-12-19T16:36:14, size: 613
Batch sent: 2024-12-19T16:36:14, size: 613
Batch sent: 2024-12-19T16:36:14, size: 354
2024-12-19T16:37:13
Data collected successfully!
 Number of flights:  10168
Batch sent: 2024-12-19T16:37:13, size: 613
Batch sent: 2024-12-19T16:37:13, size: 612
Batch sent: 2024-12-19T16:37:13